## 5. Identificar variables clave para el negocio

A continuación se seleccionan y comentan las variables que pueden tener mayor impacto en los objetivos del negocio, como duración de la partida, nivel, ranking y combinaciones de campeones.

In [ ]:
from kedro.framework.context import load_context

# Cargar contexto de Kedro (sube un nivel desde notebooks/)
context = load_context("../")
catalog = context.catalog

# Cargar datasets del catálogo
df_challenger = catalog.load("challenger_raw")
df_platinum = catalog.load("platinum_raw")
df_grandmaster = catalog.load("grandmaster_raw")


In [ ]:
for name, df in zip(['Challenger', 'Platinum', 'Grandmaster'], [df_challenger, df_platinum, df_grandmaster]):
    print(f'--- {name} ---')
    display(df.describe())
    if 'Ranked' in df.columns:
        print('Ranked value counts:')
        print(df['Ranked'].value_counts())
    if 'gameDuration' in df.columns:
        plt.figure(figsize=(8,4))
        sns.histplot(df['gameDuration'], kde=True)
        plt.title(f'Distribución de gameDuration - {name}')
        plt.show()

## 4. Visualizar estadísticas descriptivas

Analizaremos estadísticas básicas y algunas distribuciones de variables relevantes.

In [16]:
print('Challenger:')
display(df_challenger.head())
display(df_challenger.info())
print('Shape:', df_challenger.shape)

print('Platinum:')
display(df_platinum.head())
display(df_platinum.info())
print('Shape:', df_platinum.shape)

print('Grandmaster:')
display(df_grandmaster.head())
display(df_grandmaster.info())
print('Shape:', df_grandmaster.shape)

Challenger:


,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion
0,KR_4247538593,2142.470703,8,35,1,2134.272217,"{'DarkStar': 2, 'Protector': 4, 'Rebel': 1, 'S...","{'JarvanIV': {'items': [27], 'star': 3}, 'Sona..."
1,KR_4247538593,2142.470703,9,35,2,2134.272217,"{'Blaster': 2, 'Mercenary': 1, 'Rebel': 6, 'Se...","{'Malphite': {'items': [7], 'star': 2}, 'Yasuo..."
2,KR_4247538593,2142.470703,8,34,3,2073.459229,"{'Cybernetic': 1, 'DarkStar': 3, 'Demolitionis...","{'KaiSa': {'items': [99, 2, 23], 'star': 2}, '..."
3,KR_4247538593,2142.470703,8,33,4,1998.146729,"{'Blaster': 1, 'Cybernetic': 1, 'DarkStar': 1,...","{'KaiSa': {'items': [44, 37], 'star': 2}, 'Ann..."
4,KR_4247538593,2142.470703,9,33,5,1986.443237,"{'Blaster': 2, 'Demolitionist': 2, 'Mercenary'...","{'Ziggs': {'items': [], 'star': 1}, 'Yasuo': {..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79999 entries, 0 to 79998
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gameId          79999 non-null  object 
 1   gameDuration    79999 non-null  float64
 2   level           79999 non-null  int64  
 3   lastRound       79999 non-null  int64  
 4   Ranked          79999 non-null  int64  
 5   ingameDuration  79999 non-null  float64
 6   combination     79999 non-null  object 
 7   champion        79999 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 4.9+ MB


None

Shape: (79999, 8)
Platinum:


...

Shape: (80000, 8)
Grandmaster:


,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,combination,champion
0,KR_4254769102,2319.661377,9,38,1,2311.472168,"{'Blaster': 4, 'Chrono': 2, 'Cybernetic': 2, '...","{'Malphite': {'items': [], 'star': 2}, 'Blitzc..."
1,KR_4254769102,2319.661377,9,38,2,2311.472168,"{'Blaster': 2, 'Chrono': 2, 'Cybernetic': 6, '...","{'Fiora': {'items': [], 'star': 2}, 'Leona': {..."
2,KR_4254769102,2319.661377,9,37,3,2247.994873,"{'DarkStar': 2, 'Protector': 4, 'Rebel': 1, 'S...","{'JarvanIV': {'items': [], 'star': 3}, 'Sona':..."
3,KR_4254769102,2319.661377,7,33,4,1996.358643,"{'DarkStar': 2, 'Demolitionist': 1, 'Infiltrat...","{'KhaZix': {'items': [], 'star': 2}, 'KaiSa': ..."
4,KR_4254769102,2319.661377,8,31,5,1871.789551,"{'Chrono': 1, 'Cybernetic': 1, 'DarkStar': 3, ...","{'Leona': {'items': [], 'star': 1}, 'Mordekais..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gameId          80000 non-null  object 
 1   gameDuration    80000 non-null  float64
 2   level           80000 non-null  int64  
 3   lastRound       80000 non-null  int64  
 4   Ranked          80000 non-null  int64  
 5   ingameDuration  80000 non-null  float64
 6   combination     80000 non-null  object 
 7   champion        80000 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 4.9+ MB


None

Shape: (80000, 8)


## 3. Explorar estructura de los datos

Revisaremos las primeras filas, tipos de datos y dimensiones de cada dataset.

In [15]:
df_challenger = pd.read_csv('./data/01_raw/TFT_Challenger_MatchData.csv')
df_platinum = pd.read_csv('./data/01_raw/TFT_Platinum_MatchData.csv')
df_grandmaster = pd.read_csv('./data/01_raw/TFT_Grandmaster_MatchData.csv')

## 2. Cargar datos iniciales

Cargaremos los tres archivos CSV principales del proyecto para comenzar la exploración.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Importar librerías necesarias

Importaremos las librerías principales para el análisis exploratorio de datos.

# Comprensión del Negocio

Este notebook tiene como objetivo entender el contexto y los datos del proyecto TFT, comenzando por la exploración inicial de los archivos CSV disponibles.